In [3]:
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup
from bs4 import Tag
import re
import csv
import json

In [4]:
with open('crimegrade.html') as f:
    data = f.read().rstrip()

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [6]:
soup = bs(data, "html.parser")
content = soup.find("body")

In [5]:
content =content.text
type(content)

str

In [6]:
content = content.replace("\n", " ")

In [7]:
content = ' '.join(content.split())

In [8]:
content = content.split(" ")

In [9]:
len(content)

41961

In [10]:
content = content[1:]
len(content)

41960

In [11]:
def ZipAndCity(content):
    addresses = []
    for element in content:
        address = element.split(",")
        zipcode = address[0]
        city = address[1]
        newAddress = {"Zipcode": zipcode, "City": city}
        addresses.append(newAddress)
    return addresses

In [12]:
addresses = ZipAndCity(content)
addresses[:5]

[{'Zipcode': '16820', 'City': 'aaronsburg-pa'},
 {'Zipcode': '36310', 'City': 'abbeville-al'},
 {'Zipcode': '31001', 'City': 'abbeville-ga'},
 {'Zipcode': '70510', 'City': 'abbeville-la'},
 {'Zipcode': '70511', 'City': 'abbeville-la'}]

In [13]:
addresses[0]


{'Zipcode': '16820', 'City': 'aaronsburg-pa'}

In [14]:
def findUrl(addresses):
    totalUrls = []
    for element in addresses:
        city = element["City"]
        url = "https://crimegrade.org/safest-places-in-" + city + "/"
        totalUrls.append(url)
    return totalUrls

In [15]:
def findSingleUrl(city):
    url = "https://crimegrade.org/safest-places-in-" + city + "/"
    return url

In [16]:
totalUrls = findUrl(addresses)

In [17]:
totalUrls[:10]

['https://crimegrade.org/safest-places-in-aaronsburg-pa/',
 'https://crimegrade.org/safest-places-in-abbeville-al/',
 'https://crimegrade.org/safest-places-in-abbeville-ga/',
 'https://crimegrade.org/safest-places-in-abbeville-la/',
 'https://crimegrade.org/safest-places-in-abbeville-la/',
 'https://crimegrade.org/safest-places-in-abbeville-ms/',
 'https://crimegrade.org/safest-places-in-abbeville-sc/',
 'https://crimegrade.org/safest-places-in-abbot-me/',
 'https://crimegrade.org/safest-places-in-abbotsford-wi/',
 'https://crimegrade.org/safest-places-in-abbottstown-pa/']

In [18]:
def generate_soup(url):
    response = requests.get(url)
    html = response.text
    return BeautifulSoup(html, "html.parser")

In [19]:
sampleUrl = "https://crimegrade.org/safest-places-in-abbot-me/"

In [20]:
from urllib.request import Request, urlopen
req = Request(sampleUrl, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [21]:
def generate_soup(url):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, "html.parser")
        return soup
    except Exception as e:
        print(url, e)

In [22]:
scores = []
dict1 = {}
content1 = soup.find('div', {'class': 'one_half'})
grade = content1.find('p', {'class': 'overallGradeLetter gradeBTxt'}).text
overall = "Overall Crime Grade"
rows = content1.findAll('tr')
for element in rows:
    if len(element.findAll('a'))>= 1:
        crimeType = element.find('a').text
    else:
        crimeType = element.find('div', {'class': 'mtr-cell-content'}).text
    print(crimeType)
    crimeGrade = element.find('span').text
    print(crimeGrade)
    dict2 = {'CrimeType': crimeType, 'Grade':crimeGrade}
    scores.append(dict2)

Violent Crime Grade
A
Property Crime Grade
C
Other Crime Grade
A


In [23]:
def findScores(soup):
    scores = []
    content1 = soup.find('div', {'class': 'one_half'})
    grades = content1.select("p[class^=overallGradeLetter]")
    if len(grades) == 1:
        grade = grades[0].text
    overall = "Overall Crime Grade"
    dict1 = {'CrimeType': overall, 'Grade':grade}
    scores.append(dict1)
    rows = content1.findAll('tr')
    for element in rows:
        if len(element.findAll('a'))>= 1:
            crimeType = element.find('a').text
        else:
            crimeType = element.find('div', {'class': 'mtr-cell-content'}).text
        crimeGrade = element.find('span').text
        dict2 = {'CrimeType': crimeType, 'Grade':crimeGrade}
        scores.append(dict2)
    return scores

In [24]:
scores = findScores(soup)

In [25]:
scores

[{'CrimeType': 'Overall Crime Grade', 'Grade': 'B'},
 {'CrimeType': 'Violent Crime Grade', 'Grade': 'A'},
 {'CrimeType': 'Property Crime Grade', 'Grade': 'C'},
 {'CrimeType': 'Other Crime Grade', 'Grade': 'A'}]

In [26]:
tables = soup.findAll("div", {'class': 'one_third'})
detailedScores= []
summaryScores = []
for x in range(len(tables)):
    rows = tables[x].findAll('tr')
    if x < 2:
        rows = rows[1:-1]
    else:
        rows = rows[1:]
    for k in range(len(rows)):
        if k < len(rows) - 1:
            cells = rows[k].findAll('td')
            crimeType = cells[0].find('div', {'class': 'mtr-cell-content'}).text
            crimeGrade = cells[1].find('div', {'class': 'mtr-cell-content'}).text
            grades = {"CrimeType": crimeType, 'CrimeRate': crimeGrade}
            detailedScores.append(grades)
        else:
            cells = rows[k].findAll('td')
            crimeType = cells[0].find('div', {'class': 'mtr-cell-content'}).text
            crimeGrade = cells[1].find('div', {'class': 'mtr-cell-content'}).text
            crimeRate = crimeGrade.split(" ")
            rate = crimeRate[0]
            grade = crimeRate[1][1]
            grades = {"CrimeType": crimeType, 'CrimeRate': rate, 'CrimeGrade': grade}
            summaryScores.append(grades)
            
detailedScores

[{'CrimeType': 'Assault', 'CrimeRate': '0.0249'},
 {'CrimeType': 'Robbery', 'CrimeRate': '0.6940'},
 {'CrimeType': 'Rape', 'CrimeRate': '0.3188'},
 {'CrimeType': 'Murder', 'CrimeRate': '0.0151'},
 {'CrimeType': 'Theft', 'CrimeRate': '11.43'},
 {'CrimeType': 'Vehicle Theft', 'CrimeRate': '0.7051'},
 {'CrimeType': 'Burglary', 'CrimeRate': '3.792'},
 {'CrimeType': 'Arson', 'CrimeRate': '0.0451'},
 {'CrimeType': 'Kidnapping', 'CrimeRate': '0.1427'},
 {'CrimeType': 'Drug Crimes', 'CrimeRate': '0.1344'},
 {'CrimeType': 'Vandalism', 'CrimeRate': '2.751'},
 {'CrimeType': 'Identity Theft', 'CrimeRate': '1.361'},
 {'CrimeType': 'Animal Cruelty', 'CrimeRate': '0.0247'}]

In [27]:
summaryScores

[{'CrimeType': 'Total Violent Crime', 'CrimeRate': '1.053', 'CrimeGrade': 'A'},
 {'CrimeType': 'Total Property Crime',
  'CrimeRate': '15.97',
  'CrimeGrade': 'C'},
 {'CrimeType': 'Total “Other” Rate', 'CrimeRate': '4.413', 'CrimeGrade': 'A'}]

In [28]:
def findDetailedScores(soup):
    tables = soup.findAll("div", {'class': 'one_third'})
    detailedScores= []
    for x in range(len(tables)):
        rows = tables[x].findAll('tr')
        if x < 2:
            rows = rows[1:-1]
        else:
            rows = rows[1:]
        for k in range(len(rows)):
            if k < len(rows) - 1:
                cells = rows[k].findAll('td')
                crimeType = cells[0].find('div', {'class': 'mtr-cell-content'}).text
                crimeGrade = cells[1].find('div', {'class': 'mtr-cell-content'}).text
                grades = {"CrimeType": crimeType, 'CrimeRate': crimeGrade}
                detailedScores.append(grades)
    return detailedScores

In [29]:
def findSummaryScores(soup):
    tables = soup.findAll("div", {'class': 'one_third'})
    summaryScores = []
    for x in range(len(tables)):
        rows = tables[x].findAll('tr')
        if x < 2:
            rows = rows[1:-1]
        else:
            rows = rows[1:]
        for k in range(len(rows)):
            if k == len(rows) - 1:
                cells = rows[k].findAll('td')
                crimeType = cells[0].find('div', {'class': 'mtr-cell-content'}).text
                crimeGrade = cells[1].find('div', {'class': 'mtr-cell-content'}).text
                crimeRate = crimeGrade.split(" ")
                rate = crimeRate[0]
                grade = crimeRate[1][1]
                grades = {"CrimeType": crimeType, 'CrimeRate': rate, 'CrimeGrade': grade}
                summaryScores.append(grades)
    return summaryScores

In [30]:
scores

[{'CrimeType': 'Overall Crime Grade', 'Grade': 'B'},
 {'CrimeType': 'Violent Crime Grade', 'Grade': 'A'},
 {'CrimeType': 'Property Crime Grade', 'Grade': 'C'},
 {'CrimeType': 'Other Crime Grade', 'Grade': 'A'}]

In [31]:
summaryScores

[{'CrimeType': 'Total Violent Crime', 'CrimeRate': '1.053', 'CrimeGrade': 'A'},
 {'CrimeType': 'Total Property Crime',
  'CrimeRate': '15.97',
  'CrimeGrade': 'C'},
 {'CrimeType': 'Total “Other” Rate', 'CrimeRate': '4.413', 'CrimeGrade': 'A'}]

In [1]:
def updateSummaryScores(scores, summaryScores):
    overallRate = 0
    for element in summaryScores:
        grade = element['CrimeRate']
        if ',' in grade:
            grade = grade.replace(',', '')
        overallRate = overallRate + float(grade)

    overall = {'CrimeType': "Total Crime", 'CrimeRate': str(overallRate), 'CrimeGrade': scores[0]['Grade']}
    summaryScores.append(overall)
    return summaryScores

In [33]:
newSummaryScores = updateSummaryScores(scores, summaryScores)
newSummaryScores

[{'CrimeType': 'Total Violent Crime', 'CrimeRate': '1.053', 'CrimeGrade': 'A'},
 {'CrimeType': 'Total Property Crime',
  'CrimeRate': '15.97',
  'CrimeGrade': 'C'},
 {'CrimeType': 'Total “Other” Rate', 'CrimeRate': '4.413', 'CrimeGrade': 'A'},
 {'CrimeType': 'Total Crime', 'CrimeRate': '21.436', 'CrimeGrade': 'B'}]

In [34]:
detailedScores[0]["CrimeType"]

'Assault'

In [35]:
dict2 = {}
for element in detailedScores:
    key = element["CrimeType"]
    dict2[key] = element["CrimeRate"]
for element in newSummaryScores:
    dict2[element["CrimeType"]] = element["CrimeRate"]
    newKey = element["CrimeType"] + " " + "Grade"
    dict2[newKey] = element["CrimeGrade"]
    
dict2

{'Assault': '0.0249',
 'Robbery': '0.6940',
 'Rape': '0.3188',
 'Murder': '0.0151',
 'Theft': '11.43',
 'Vehicle Theft': '0.7051',
 'Burglary': '3.792',
 'Arson': '0.0451',
 'Kidnapping': '0.1427',
 'Drug Crimes': '0.1344',
 'Vandalism': '2.751',
 'Identity Theft': '1.361',
 'Animal Cruelty': '0.0247',
 'Total Violent Crime': '1.053',
 'Total Violent Crime Grade': 'A',
 'Total Property Crime': '15.97',
 'Total Property Crime Grade': 'C',
 'Total “Other” Rate': '4.413',
 'Total “Other” Rate Grade': 'A',
 'Total Crime': '21.436',
 'Total Crime Grade': 'B'}

In [36]:
def findTotalScores(soup):
    scores = findScores(soup)
    detailedScores = findDetailedScores(soup)
    summaryScores = findSummaryScores(soup)
    newSummaryScores = updateSummaryScores(scores, summaryScores)
    dict2 = {}
    for element in detailedScores:
        key = element["CrimeType"]
        dict2[key] = element["CrimeRate"]
    for element in newSummaryScores:
        dict2[element["CrimeType"]] = element["CrimeRate"]
        newKey = element["CrimeType"] + " " + "Grade"
        dict2[newKey] = element["CrimeGrade"]
    return dict2

In [37]:
totalUrls[:5]

['https://crimegrade.org/safest-places-in-aaronsburg-pa/',
 'https://crimegrade.org/safest-places-in-abbeville-al/',
 'https://crimegrade.org/safest-places-in-abbeville-ga/',
 'https://crimegrade.org/safest-places-in-abbeville-la/',
 'https://crimegrade.org/safest-places-in-abbeville-la/']

In [38]:
addresses[:5]

[{'Zipcode': '16820', 'City': 'aaronsburg-pa'},
 {'Zipcode': '36310', 'City': 'abbeville-al'},
 {'Zipcode': '31001', 'City': 'abbeville-ga'},
 {'Zipcode': '70510', 'City': 'abbeville-la'},
 {'Zipcode': '70511', 'City': 'abbeville-la'}]

In [39]:
addressList = []
for element in addresses:
    address = element['City']
    addressList.append(address)

In [40]:
def generateData(addresses):
    data = []
    for address in addresses:
        zipcode = address["Zipcode"]
        city = address["City"]
        url = findSingleUrl(city)
        soup = generate_soup(url)
        if soup != None:
            dict2 = findTotalScores(soup)
            dict2["Zipcode"] = zipcode
            dict2["City"] = city
            data.append(dict2)
    return data

In [41]:
len(addresses)

41960

In [122]:
list1 = addresses[:2000]
list2 = addresses[:4000]

In [123]:
list1 = addresses[:1000]

In [124]:
data1 = generateData(list1)
data1[:3]

[{'Assault': '0.3893',
  'Robbery': '0.0751',
  'Rape': '0.3315',
  'Murder': '0.0451',
  'Theft': '3.027',
  'Vehicle Theft': '0.7065',
  'Burglary': '3.127',
  'Arson': '0.0110',
  'Kidnapping': '0.2651',
  'Drug Crimes': '0.4330',
  'Vandalism': '3.829',
  'Identity Theft': '1.514',
  'Animal Cruelty': '0.0063',
  'Total Violent Crime': '0.8410',
  'Total Violent Crime Grade': 'A',
  'Total Property Crime': '6.872',
  'Total Property Crime Grade': 'A',
  'Total “Other” Rate': '6.047',
  'Total “Other” Rate Grade': 'B',
  'Total Crime': '13.76',
  'Total Crime Grade': 'A',
  'Zipcode': '16820',
  'City': 'aaronsburg-pa'},
 {'Assault': '3.300',
  'Robbery': '0.7257',
  'Rape': '0.4133',
  'Murder': '0.0609',
  'Theft': '10.81',
  'Vehicle Theft': '0.5586',
  'Burglary': '9.105',
  'Arson': '0.0689',
  'Kidnapping': '0.1354',
  'Drug Crimes': '5.078',
  'Vandalism': '17.12',
  'Identity Theft': '0.7516',
  'Animal Cruelty': '0.1409',
  'Total Violent Crime': '4.500',
  'Total Violent C

In [43]:
col_names = ['Assault','Robbery','Rape','Murder','Theft','Vehicle Theft','Burglary','Arson','Kidnapping',
             'Drug Crimes','Vandalism','Identity Theft','Animal Cruelty','Total Violent Crime',
             'Total Violent Crime Grade','Total Property Crime','Total Property Crime Grade','Total “Other” Rate',
             'Total “Other” Rate Grade','Total Crime','Total Crime Grade','Zipcode','City']

In [126]:
with open("CrimeGrade_1.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data1:
            wr.writerow(ele)

In [69]:
list2 = addresses[1000:2000]
data2 = generateData(list2)
data2[:2]

[{'Assault': '0.6189',
  'Robbery': '0.6106',
  'Rape': '0.4312',
  'Murder': '0.0153',
  'Theft': '8.330',
  'Vehicle Theft': '0.8118',
  'Burglary': '2.860',
  'Arson': '0.0545',
  'Kidnapping': '0.0316',
  'Drug Crimes': '0.6338',
  'Vandalism': '3.983',
  'Identity Theft': '1.094',
  'Animal Cruelty': '0.0330',
  'Total Violent Crime': '1.676',
  'Total Violent Crime Grade': 'B',
  'Total Property Crime': '12.06',
  'Total Property Crime Grade': 'B',
  'Total “Other” Rate': '5.775',
  'Total “Other” Rate Grade': 'B',
  'Total Crime': '19.511000000000003',
  'Total Crime Grade': 'B+',
  'Zipcode': '55302',
  'City': 'annandale-mn'},
 {'Assault': '0.1056',
  'Robbery': '0.6788',
  'Rape': '0.1389',
  'Murder': '0.0189',
  'Theft': '1.904',
  'Vehicle Theft': '0.1685',
  'Burglary': '1.153',
  'Arson': '0.0063',
  'Kidnapping': '0.0799',
  'Drug Crimes': '2.867',
  'Vandalism': '0.8332',
  'Identity Theft': '1.295',
  'Animal Cruelty': '0.0117',
  'Total Violent Crime': '0.9421',
  'T

In [70]:
with open("CrimeGrade_2.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data2:
            wr.writerow(ele)

In [71]:
list3 = addresses[2000:3000]
data3 = generateData(list3)
with open("CrimeGrade_3.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data3:
            wr.writerow(ele)

In [73]:
list4 = addresses[3000:4000]
data4 = generateData(list4)
with open("CrimeGrade_4.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data4:
            wr.writerow(ele)

In [74]:
list5 = addresses[4000:5000]
data5 = generateData(list5)
with open("CrimeGrade_5.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data5:
            wr.writerow(ele)

In [75]:
list6 = addresses[5000:6000]
data6 = generateData(list6)
with open("CrimeGrade_6.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data6:
            wr.writerow(ele)

In [78]:
list7 = addresses[6000:7000]
data7 = generateData(list7)
with open("CrimeGrade_7.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data7:
            wr.writerow(ele)

In [77]:
list8 = addresses[7000:8000]
data8 = generateData(list8)
with open("CrimeGrade_8.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data8:
            wr.writerow(ele)

In [127]:
list9 = addresses[8000:9000]
data9 = generateData(list9)
with open("CrimeGrade_9.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data9:
            wr.writerow(ele)

In [128]:
list10 = addresses[9000:10000]
data10 = generateData(list10)
with open("CrimeGrade_10.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data10:
            wr.writerow(ele)

In [158]:
list11 = addresses[10000:11000]
data11 = generateData(list11)
with open("CrimeGrade_11.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data11:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-east-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-pittsburgh-pa/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-providence-ri/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-rochester-ny/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-east-syracuse-ny/ HTTP Error 404: Not Found


In [132]:
list12 = addresses[11000:12000]
data12 = generateData(list12)
with open("CrimeGrade_12.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data12:
            wr.writerow(ele)

In [133]:
list13 = addresses[12000:13000]
data13 = generateData(list13)
with open("CrimeGrade_13.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data13:
            wr.writerow(ele)

In [134]:
list14 = addresses[13000:14000]
data14 = generateData(list14)
with open("CrimeGrade_14.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data14:
            wr.writerow(ele)

In [136]:
list15 = addresses[14000:15000]
data15 = generateData(list15)
with open("CrimeGrade_15.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data15:
            wr.writerow(ele)

In [137]:
list16 = addresses[15000:16000]
data16 = generateData(list16)
with open("CrimeGrade_16.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data16:
            wr.writerow(ele)

In [138]:
list17 = addresses[16000:17000]
data17 = generateData(list17)
with open("CrimeGrade_17.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data17:
            wr.writerow(ele)

In [139]:
list18 = addresses[17000:18000]
data18 = generateData(list18)
with open("CrimeGrade_18.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data18:
            wr.writerow(ele)

In [140]:
list19 = addresses[18000:19000]
data19 = generateData(list19)
with open("CrimeGrade_19.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data19:
            wr.writerow(ele)

In [159]:
list20 = addresses[19000:20000]
data20 = generateData(list20)
with open("CrimeGrade_20.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data20:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-lake-dallas-tx/ HTTP Error 404: Not Found


In [142]:
list21 = addresses[20000:21000]
data21 = generateData(list21)
with open("CrimeGrade_21.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data21:
            wr.writerow(ele)

In [143]:
list22 = addresses[21000:22000]
data22 = generateData(list22)
with open("CrimeGrade_22.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data22:
            wr.writerow(ele)

In [144]:
list23 = addresses[22000:23000]
data23 = generateData(list23)
with open("CrimeGrade_23.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data23:
            wr.writerow(ele)

In [145]:
list24 = addresses[23000:24000]
data24 = generateData(list24)
with open("CrimeGrade_24.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data24:
            wr.writerow(ele)

In [146]:
list25 = addresses[24000:25000]
data25 = generateData(list25)
with open("CrimeGrade_25.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data25:
            wr.writerow(ele)

In [44]:
list26 = addresses[25000:26000]
data26 = generateData(list26)
with open("CrimeGrade_26.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data26:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-new-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-new-philadelphia-pa/ HTTP Error 404: Not Found


In [45]:
list27 = addresses[26000:27000]
data27 = generateData(list27)
with open("CrimeGrade_27.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data27:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-north-charleston-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-charleston-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-charleston-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-charleston-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-charleston-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-chicago-il/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-chicago-il/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-fort-myers-fl/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-fort-myers-fl/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-fort-myers-fl/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-las-vegas-nv/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-north-las-

In [46]:
list28 = addresses[27000:28000]
data28 = generateData(list28)
with open("CrimeGrade_28.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data28:
            wr.writerow(ele)

In [47]:
list29 = addresses[28000:29000]
data29 = generateData(list29)
with open("CrimeGrade_29.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data29:
            wr.writerow(ele)

In [48]:
list30 = addresses[29000:30000]
data30 = generateData(list30)
with open("CrimeGrade_30.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data30:
            wr.writerow(ele)

In [49]:
list31 = addresses[30000:31000]
data31 = generateData(list31)
with open("CrimeGrade_31.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data31:
            wr.writerow(ele)

In [50]:
list32 = addresses[31000:32000]
data32 = generateData(list32)
with open("CrimeGrade_32.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data32:
            wr.writerow(ele)

In [51]:
list33 = addresses[32000:33000]
data33 = generateData(list33)
with open("CrimeGrade_33.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data33:
            wr.writerow(ele)

In [52]:
list34 = addresses[33000:34000]
data34 = generateData(list34)
with open("CrimeGrade_34.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data34:
            wr.writerow(ele)

In [53]:
list35 = addresses[34000:35000]
data35 = generateData(list35)
with open("CrimeGrade_35.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data35:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-south-houston-tx/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-south-milwaukee-wi/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-south-san-francisco-ca/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-south-san-francisco-ca/ HTTP Error 404: Not Found


In [54]:
list36 = addresses[35000:36000]
data36 = generateData(list36)
with open("CrimeGrade_36.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data36:
            wr.writerow(ele)

In [55]:
list37 = addresses[36000:37000]
data37 = generateData(list37)
with open("CrimeGrade_37.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data37:
            wr.writerow(ele)

In [59]:
list38 = addresses[37000:38000]
data38 = generateData(list38)
with open("CrimeGrade_38.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data38:
            wr.writerow(ele)

In [60]:
list39 = addresses[38000:39000]
data39 = generateData(list39)
with open("CrimeGrade_39.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data39:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-west-chicago-il/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-chicago-il/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-columbia-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-columbia-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-columbia-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-columbia-sc/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-des-moines-ia/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-des-moines-ia/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-hartford-ct/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-west-hartford-ct/ HTTP Error 404: 

In [61]:
list40 = addresses[39000:40000]
data40 = generateData(list40)
with open("CrimeGrade_40.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data40:
            wr.writerow(ele)

In [62]:
list41 = addresses[40000:41000]
data41 = generateData(list41)
with open("CrimeGrade_41.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data41:
            wr.writerow(ele)

In [63]:
list42 = addresses[41000:]
data42 = generateData(list42)
with open("CrimeGrade_42.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data42:
            wr.writerow(ele)

https://crimegrade.org/safest-places-in-lake-st-louis-mo/ HTTP Error 404: Not Found


In [64]:
addresses[:10]

[{'Zipcode': '16820', 'City': 'aaronsburg-pa'},
 {'Zipcode': '36310', 'City': 'abbeville-al'},
 {'Zipcode': '31001', 'City': 'abbeville-ga'},
 {'Zipcode': '70510', 'City': 'abbeville-la'},
 {'Zipcode': '70511', 'City': 'abbeville-la'},
 {'Zipcode': '38601', 'City': 'abbeville-ms'},
 {'Zipcode': '29620', 'City': 'abbeville-sc'},
 {'Zipcode': '04406', 'City': 'abbot-me'},
 {'Zipcode': '54405', 'City': 'abbotsford-wi'},
 {'Zipcode': '17301', 'City': 'abbottstown-pa'}]

In [77]:
def findZipUrl(zipcode):
    url = "https://crimegrade.org/safest-places-in-" + zipcode + "/"
    return url

In [74]:
def generate_soup2(url):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, "html.parser")
        return soup
    except Exception as e:
        print(url, e)

In [70]:
def generateData2(addresses):
    data = []
    for address in addresses:
        zipcode = address["Zipcode"]
        city = address["City"]
        url = findSingleUrl(city)
        soup = generate_soup2(url)
        if soup == None:
            url = findZipUrl(zipcode)
            soup = generate_soup(url)
            dict2 = findTotalScores(soup)
            dict2["Zipcode"] = zipcode
            dict2["City"] = city
            data.append(dict2)
    return data

In [72]:
list11 = addresses[10000:11000]

In [81]:
add = ["east-harford-ct", "east-pittsburgh-pa", "east-providence-ri", "east-rochester-ny", "north-syracuse-ny",
                "north-charleson-sc","north-chicago-il", "north-fort-meyers-fl", "north-las-vegas-nv", "north-little-rock-ar",
                "north-providence-ri", "south-houston-tx", "south-milewaukee-wi", "south-san-francisco-ca",
                "west-columbia-sc", "west-des-moines-ia", "west-hartford-ct", "west-sacramento-ca", "west-chicago-il"]

In [84]:
data = []
for address in addresses:
    city = address["City"]
    zipcode = address["Zipcode"]
    for element in add:
        if city == element:
            url = findZipUrl(zipcode)
            soup = generate_soup2(url)
            if soup != None:
                dict2 = findTotalScores(soup)
                dict2["Zipcode"] = zipcode
                dict2["City"] = city
                data.append(dict2)

https://crimegrade.org/safest-places-in-60086/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-89033/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-89036/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-89087/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-72115/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-72124/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-72190/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-72199/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-72198/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-94083/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-60186/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-29171/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-06127/ HTTP Error 404: Not Found
https://crimegrade.org/safest-places-in-06133/ HTTP

In [87]:
with open("CrimeGrade_43.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in data:
            wr.writerow(ele)